In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
!python -m pip install -U pip wheel
%pip install -U ipython
%pip install -U so-vits-svc-fork

from IPython.display import clear_output
clear_output()

!mkdir -p "dataset_raw_raw"
!mkdir -p "dataset_raw"

In [ ]:
DATASET_NAME = "raffaella" #@param ["felipevi", "heroes", "mecano", "raffaella"]
F0_METHOD = "dio" #param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]

In [ ]:
split = False

if split:
  !cp -R /content/drive/MyDrive/so-vits-svc-fork/dataset_raw_raw/{DATASET_NAME}/ -t "dataset_raw_raw/"
  !svc pre-split
  !zip {DATASET_NAME}_raw.zip dataset_raw/{DATASET_NAME}/*
  files.download(f"{DATASET_NAME}_raw.zip")
else:
  !cp -R /content/drive/MyDrive/so-vits-svc-fork/dataset_raw/{DATASET_NAME}/ -t "dataset_raw/"

!svc pre-resample
!svc pre-config
!svc pre-hubert -fm {F0_METHOD}

In [ ]:
!cp -R configs/44k/config.json drive/MyDrive/so-vits-svc-fork

# Entrenamiento: Ojo, que machaca el anterior

In [ ]:
%load_ext tensorboard
%tensorboard --logdir drive/MyDrive/so-vits-svc-fork/logs/{DATASET_NAME}/44k
!svc train --model-path drive/MyDrive/so-vits-svc-fork/logs/{DATASET_NAME}/44k

## Inferencia

In [ ]:
from IPython.display import Audio, display

In [ ]:
NAME = "original_3" #@param ["original_1", "original_2", "original_3", "original_4", "original_5", "original_6", "original_7", "original_8"]
F0_METHOD = "crepe" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]
TRANSPOSE = 0 #@param [-12,0,12]


!svc infer drive/MyDrive/so-vits-svc-fork/audio/{NAME}.wav -na -fm {F0_METHOD} -t {TRANSPOSE} -o drive/MyDrive/so-vits-svc-fork/audio/{NAME}.{DATASET_NAME}.wav -m drive/MyDrive/so-vits-svc-fork/logs/{DATASET_NAME}/44k/ -c drive/MyDrive/so-vits-svc-fork/logs/{DATASET_NAME}/44k/config.json
display(Audio(f"drive/MyDrive/so-vits-svc-fork/audio/{NAME}.{DATASET_NAME}.wav", autoplay=True))